# Home Credit Default Risk

Can you predict how capable each applicant is of repaying a loan?

Many people struggle to get loans due to **insufficient or non-existent credit histories**. And, unfortunately, this population is often taken advantage of by untrustworthy lenders.

Home Credit strives to broaden financial inclusion for the **unbanked population by providing a positive and safe borrowing experience**. In order to make sure this underserved population has a positive loan experience, Home Credit makes use of a variety of alternative data--including telco and transactional information--to predict their clients' repayment abilities.

While Home Credit is currently using various statistical and machine learning methods to make these predictions, they're challenging Kagglers to help them unlock the full potential of their data. Doing so will ensure that clients capable of repayment are not rejected and that loans are given with a principal, maturity, and repayment calendar that will empower their clients to be successful.

**Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.**

# Dataset

In [915]:
# #Python Libraries
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels
import pandas_profiling

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import time
import random
import requests
import datetime

import missingno as msno
import math
import sys
import gc
import os

# #sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor

# #sklearn - preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# #sklearn - metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score

# #XGBoost & LightGBM
import xgboost as xgb
import lightgbm as lgb

# #Missing value imputation
from fancyimpute import KNN, MICE

# #Hyperparameter Optimization
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

# #MongoDB for Model Parameter Storage
from pymongo import MongoClient

pd.options.display.max_columns = 150

## Data Dictionary

In [916]:
!ls -l ../data/

total 2621364
-rw-r--r-- 1 karti 197609  26567651 May 17 18:06 application_test.csv
-rw-r--r-- 1 karti 197609 166133370 May 17 18:06 application_train.csv
-rw-r--r-- 1 karti 197609 170016717 May 17 18:08 bureau.csv
-rw-r--r-- 1 karti 197609 375592889 May 17 18:08 bureau_balance.csv
-rw-r--r-- 1 karti 197609 424582605 May 17 18:10 credit_card_balance.csv
-rw-r--r-- 1 karti 197609     37436 May 30 00:41 HomeCredit_columns_description.csv
-rw-r--r-- 1 karti 197609 723118349 May 17 18:13 installments_payments.csv
-rw-r--r-- 1 karti 197609 392703158 May 17 18:14 POS_CASH_balance.csv
-rw-r--r-- 1 karti 197609 404973293 May 17 18:15 previous_application.csv
-rw-r--r-- 1 karti 197609    536202 May 17 18:06 sample_submission.csv


- application_{train|test}.csv

This is the main table, broken into two files for Train (**with TARGET**) and Test (without TARGET).
Static data for all applications. **One row represents one loan in our data sample.**

Observations:
* Each row is unique

-----


- bureau.csv

All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

- bureau_balance.csv

Monthly balances of previous credits in Credit Bureau.
This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

- POS_CASH_balance.csv

Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

- credit_card_balance.csv

Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

-----

- previous_application.csv

All previous applications for Home Credit loans of clients who have loans in our sample.
There is one row for each previous application related to loans in our data sample.

-----

- installments_payments.csv

Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
There is a) one row for every payment that was made plus b) one row each for missed payment.
One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.

- HomeCredit_columns_description.csv

This file contains descriptions for the columns in the various data files.

![](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

# Data Pre-processing

In [917]:
df_application_train_original = pd.read_csv("../data/application_train.csv")
df_application_test_original = pd.read_csv("../data/application_test.csv")
df_bureau_original = pd.read_csv("../data/bureau.csv")
df_bureau_balance_original = pd.read_csv("../data/bureau_balance.csv")
df_credit_card_balance_original = pd.read_csv("../data/credit_card_balance.csv")
df_installments_payments_original = pd.read_csv("../data/installments_payments.csv")
df_pos_cash_balance_original = pd.read_csv("../data/POS_CASH_balance.csv")
df_previous_application_original = pd.read_csv("../data/previous_application.csv")

In [918]:
df_application_train = pd.read_csv("../data/application_train.csv")
df_application_test = pd.read_csv("../data/application_test.csv")
df_bureau = pd.read_csv("../data/bureau.csv")
df_bureau_balance = pd.read_csv("../data/bureau_balance.csv")
df_credit_card_balance = pd.read_csv("../data/credit_card_balance.csv")
df_installments_payments = pd.read_csv("../data/installments_payments.csv")
df_pos_cash_balance = pd.read_csv("../data/POS_CASH_balance.csv")
df_previous_application = pd.read_csv("../data/previous_application.csv")

In [919]:
print("df_application_train: ", df_application_train.shape)
print("df_application_test: ", df_application_test.shape)
print("df_bureau: ", df_bureau.shape)
print("df_bureau_balance: ", df_bureau_balance.shape)
print("df_credit_card_balance: ", df_credit_card_balance.shape)
print("df_installments_payments: ", df_installments_payments.shape)
print("df_pos_cash_balance: ", df_pos_cash_balance.shape)
print("df_previous_application: ", df_previous_application.shape)

df_application_train:  (307511, 122)
df_application_test:  (48744, 121)
df_bureau:  (1716428, 17)
df_bureau_balance:  (27299925, 3)
df_credit_card_balance:  (3840312, 23)
df_installments_payments:  (13605401, 8)
df_pos_cash_balance:  (10001358, 8)
df_previous_application:  (1670214, 37)


In [920]:
gc.collect()

102

## Feature: df_installments_payments

In [921]:
df_installments_payments['K_PREV_INSTALLMENT_PAYMENT_COUNT'] = df_installments_payments.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')

# #Note: I haven't added the K_NUM_INSTALMENT_NUMBER_SUM feature since the K_NUM_INSTALMENT_NUMBER_SUM_TO_COUNT_RATIO
# #performed better i.e. had a higher value on the feature importance score
df_installments_payments['K_NUM_INSTALMENT_NUMBER_SUM'] = df_installments_payments.groupby('SK_ID_CURR')['NUM_INSTALMENT_NUMBER'].transform(np.sum)
df_installments_payments['K_NUM_INSTALMENT_NUMBER_SUM_TO_COUNT_RATIO'] = df_installments_payments['K_NUM_INSTALMENT_NUMBER_SUM']/df_installments_payments['K_PREV_INSTALLMENT_PAYMENT_COUNT']

df_installments_payments['TEMP_DAYS_INSTALMENT'] = df_installments_payments.groupby('SK_ID_CURR')['DAYS_INSTALMENT'].transform(np.sum)
df_installments_payments['TEMP_DAYS_ENTRY_PAYMENT'] = df_installments_payments.groupby('SK_ID_CURR')['DAYS_ENTRY_PAYMENT'].transform(np.sum)
df_installments_payments['K_INST_DAYS_DIFF'] = df_installments_payments['TEMP_DAYS_INSTALMENT'] - df_installments_payments['TEMP_DAYS_ENTRY_PAYMENT']
df_installments_payments['K_INST_DAYS_DIFF_TO_COUNT_RATIO'] = df_installments_payments['K_INST_DAYS_DIFF']/df_installments_payments['K_PREV_INSTALLMENT_PAYMENT_COUNT']

# df_installments_payments['K_DAYS_ENTRY_PAYMENT_MEAN'] = df_installments_payments.groupby('SK_ID_CURR')['DAYS_ENTRY_PAYMENT'].transform(np.mean)
df_installments_payments['K_DAYS_ENTRY_PAYMENT_MAX'] = df_installments_payments.groupby('SK_ID_CURR')['DAYS_ENTRY_PAYMENT'].transform(np.max)
# df_installments_payments['K_DAYS_ENTRY_PAYMENT_MIN'] = df_installments_payments.groupby('SK_ID_CURR')['DAYS_ENTRY_PAYMENT'].transform(np.min)
df_installments_payments['K_DAYS_ENTRY_PAYMENT_VAR'] = df_installments_payments.groupby('SK_ID_CURR')['DAYS_ENTRY_PAYMENT'].transform(np.var)

df_installments_payments['TEMP_AMT_INSTALMENT'] = df_installments_payments.groupby('SK_ID_CURR')['AMT_INSTALMENT'].transform(np.sum)
df_installments_payments['TEMP_AMT_PAYMENT'] = df_installments_payments.groupby('SK_ID_CURR')['AMT_PAYMENT'].transform(np.sum)
df_installments_payments['K_INST_AMT_DIFF'] = df_installments_payments['TEMP_AMT_INSTALMENT'] - df_installments_payments['TEMP_AMT_PAYMENT']
# #Note: I haven't added the K_INST_AMT_DIFF_TO_COUNT_RATIO feature since the K_INST_AMT_DIFF
# #performed better individually than with the count_ratio i.e. had a higher value on the feature importance score
# df_installments_payments['K_INST_AMT_DIFF_TO_COUNT_RATIO'] = df_installments_payments['K_INST_AMT_DIFF']/df_installments_payments['K_PREV_INSTALLMENT_PAYMENT_COUNT']

df_installments_payments['K_AMT_PAYMENT_MEAN'] = df_installments_payments.groupby('SK_ID_CURR')['AMT_PAYMENT'].transform(np.mean)
df_installments_payments['K_AMT_PAYMENT_MAX'] = df_installments_payments.groupby('SK_ID_CURR')['AMT_PAYMENT'].transform(np.max)
df_installments_payments['K_AMT_PAYMENT_MIN'] = df_installments_payments.groupby('SK_ID_CURR')['AMT_PAYMENT'].transform(np.min)
df_installments_payments['K_AMT_PAYMENT_VAR'] = df_installments_payments.groupby('SK_ID_CURR')['AMT_PAYMENT'].transform(np.var)

# #Drop Duplicates
df_installments_payments = df_installments_payments[['SK_ID_CURR', 'K_PREV_INSTALLMENT_PAYMENT_COUNT', 
                                                     'K_INST_DAYS_DIFF', 'K_INST_AMT_DIFF',
                                                     'K_NUM_INSTALMENT_NUMBER_SUM_TO_COUNT_RATIO', 'K_INST_DAYS_DIFF_TO_COUNT_RATIO',
                                                     'K_DAYS_ENTRY_PAYMENT_MAX',
                                                     'K_DAYS_ENTRY_PAYMENT_VAR', 
                                                     'K_AMT_PAYMENT_MEAN', 'K_AMT_PAYMENT_MAX', 'K_AMT_PAYMENT_MIN', 'K_AMT_PAYMENT_VAR'
                                                    ]].drop_duplicates()

In [922]:
# #CHECKPOINT
print("df_installments_payments", df_installments_payments.shape)
print(len(set(df_installments_payments["SK_ID_CURR"]).intersection(set(df_application_train["SK_ID_CURR"]))))
print(len(set(df_installments_payments["SK_ID_CURR"]).intersection(set(df_application_test["SK_ID_CURR"]))))
print("Sum: ", 291643 + 47944)

df_installments_payments (339587, 12)
291643
47944
Sum:  339587


In [923]:
gc.collect()

100

## Feature: df_credit_card_balance

In [924]:
df_credit_card_balance['K_PREV_CREDIT_CARD_BALANCE_COUNT'] = df_credit_card_balance.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')

# #All the four features below did not seem to have much weight in the feature importance of the model
# df_credit_card_balance['K_MONTHS_BALANCE_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.max)
# df_credit_card_balance['K_MONTHS_BALANCE_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.min)
# df_credit_card_balance['K_MONTHS_BALANCE_SUM'] = df_credit_card_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.sum)
# df_credit_card_balance['K_MONTHS_BALANCE_SUM_TO_COUNT_RATIO'] = df_credit_card_balance['K_MONTHS_BALANCE_SUM']/df_credit_card_balance['K_PREV_CREDIT_CARD_BALANCE_COUNT'] 

df_credit_card_balance['TEMP_AMT_BALANCE'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_BALANCE'].transform(lambda x:x+1)
df_credit_card_balance['TEMP_AMT_CREDIT_LIMIT_ACTUAL'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_CREDIT_LIMIT_ACTUAL'].transform(lambda x:x+1)
df_credit_card_balance['TEMP_UTILIZATION'] = df_credit_card_balance['TEMP_AMT_BALANCE']/df_credit_card_balance['TEMP_AMT_CREDIT_LIMIT_ACTUAL']
df_credit_card_balance['K_CREDIT_UTILIZATION_MEAN'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_UTILIZATION'].transform(np.mean)
df_credit_card_balance['K_CREDIT_UTILIZATION_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_UTILIZATION'].transform(np.min)
df_credit_card_balance['K_CREDIT_UTILIZATION_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_UTILIZATION'].transform(np.max)
df_credit_card_balance['K_CREDIT_UTILIZATION_VAR'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_UTILIZATION'].transform(np.var)

# #Validation: SK_ID_CURR = 105755
# #AMT_DRAWINGS_CURRENT = AMT_DRAWINGS_ATM_CURRENT + AMT_DRAWINGS_OTHER_CURRENT + AMT_DRAWINGS_POS_CURRENT
df_credit_card_balance['K_AMT_DRAWINGS_CURRENT_MEAN'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].transform(np.mean)
# df_credit_card_balance['K_AMT_DRAWINGS_CURRENT_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].transform(np.min)
df_credit_card_balance['K_AMT_DRAWINGS_CURRENT_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].transform(np.max)
# df_credit_card_balance['K_AMT_DRAWINGS_CURRENT_SUM'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].transform(np.sum)

df_credit_card_balance['TEMP_AMT_PAYMENT_TOTAL_CURRENT'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_PAYMENT_TOTAL_CURRENT'].transform(lambda x:x+1)
df_credit_card_balance['TEMP_AMT_TOTAL_RECEIVABLE'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_TOTAL_RECEIVABLE'].transform(lambda x:x+1)
df_credit_card_balance['TEMP_AMT_PAYMENT_OVER_RECEIVABLE'] = df_credit_card_balance['TEMP_AMT_PAYMENT_TOTAL_CURRENT']/df_credit_card_balance['TEMP_AMT_TOTAL_RECEIVABLE']
df_credit_card_balance['K_AMT_PAYMENT_OVER_RECEIVABLE_MEAN'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_AMT_PAYMENT_OVER_RECEIVABLE'].transform(np.mean)
df_credit_card_balance['K_AMT_PAYMENT_OVER_RECEIVABLE_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_AMT_PAYMENT_OVER_RECEIVABLE'].transform(np.min)
df_credit_card_balance['K_AMT_PAYMENT_OVER_RECEIVABLE_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_AMT_PAYMENT_OVER_RECEIVABLE'].transform(np.max)

# #CNT_DRAWINGS_CURRENT = CNT_DRAWINGS_ATM_CURRENT + CNT_DRAWINGS_OTHER_CURRENT + CNT_DRAWINGS_POS_CURRENT
df_credit_card_balance['K_CNT_DRAWINGS_CURRENT_MEAN'] = df_credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_CURRENT'].transform(np.mean)
df_credit_card_balance['K_CNT_DRAWINGS_CURRENT_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_CURRENT'].transform(np.min)
df_credit_card_balance['K_CNT_DRAWINGS_CURRENT_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_CURRENT'].transform(np.max)
# df_credit_card_balance['K_CNT_DRAWINGS_CURRENT_SUM'] = df_credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_CURRENT'].transform(np.sum)

# #Feature - CNT_INSTALMENT_MATURE_CUM
df_credit_card_balance['K_CNT_INSTALMENT_MATURE_CUM_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT_MATURE_CUM'].transform(np.max)
df_credit_card_balance['K_CNT_INSTALMENT_MATURE_CUM_MAX_TO_COUNT_RATIO'] = df_credit_card_balance['K_CNT_INSTALMENT_MATURE_CUM_MAX']/df_credit_card_balance['K_PREV_CREDIT_CARD_BALANCE_COUNT']

# #Feature - SK_DPD
# df_credit_card_balance['K_SK_DPD_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['SK_DPD'].transform(np.max)
# df_credit_card_balance['K_SK_DPD_MEAN'] = df_credit_card_balance.groupby('SK_ID_CURR')['SK_DPD'].transform(np.mean)
# df_credit_card_balance['K_SK_DPD_SUM'] = df_credit_card_balance.groupby('SK_ID_CURR')['SK_DPD'].transform(np.sum)
# df_credit_card_balance['K_SK_DPD_VAR'] = df_credit_card_balance.groupby('SK_ID_CURR')['SK_DPD'].transform(np.var)


# #Drop Duplicates
df_credit_card_balance = df_credit_card_balance[['SK_ID_CURR', 'K_PREV_CREDIT_CARD_BALANCE_COUNT',
                    'K_CREDIT_UTILIZATION_MEAN', 'K_CREDIT_UTILIZATION_MIN', 'K_CREDIT_UTILIZATION_MAX', 'K_CREDIT_UTILIZATION_VAR',
                    'K_AMT_DRAWINGS_CURRENT_MEAN', 'K_AMT_DRAWINGS_CURRENT_MAX', 
                    'K_AMT_PAYMENT_OVER_RECEIVABLE_MEAN', 'K_AMT_PAYMENT_OVER_RECEIVABLE_MIN', 'K_AMT_PAYMENT_OVER_RECEIVABLE_MAX', 
                    'K_CNT_DRAWINGS_CURRENT_MEAN', 'K_CNT_DRAWINGS_CURRENT_MIN', 'K_CNT_DRAWINGS_CURRENT_MAX',
                    'K_CNT_INSTALMENT_MATURE_CUM_MAX_TO_COUNT_RATIO']].drop_duplicates()

In [925]:
# #CHECKPOINT
print("df_credit_card_balance", df_credit_card_balance.shape)
print(len(set(df_credit_card_balance["SK_ID_CURR"]).intersection(set(df_application_train["SK_ID_CURR"]))))
print(len(set(df_credit_card_balance["SK_ID_CURR"]).intersection(set(df_application_test["SK_ID_CURR"]))))
print("Sum: ", 86905 + 16653)

df_credit_card_balance (103558, 15)
86905
16653
Sum:  103558


In [926]:
gc.collect()

110

## Feature: df_pos_cash_balance

In [927]:
df_pos_cash_balance['K_PREV_POS_CASH_BALANCE_COUNT'] = df_pos_cash_balance.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')

df_pos_cash_balance['K_MONTHS_BALANCE_POS_CASH_MEAN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.mean)
df_pos_cash_balance['K_MONTHS_BALANCE_POS_CASH_MAX'] = df_pos_cash_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.max)
df_pos_cash_balance['K_MONTHS_BALANCE_POS_CASH_MIN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.min)

df_pos_cash_balance['K_CNT_INSTALMENT_MEAN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT'].transform(np.mean)
df_pos_cash_balance['K_CNT_INSTALMENT_MAX'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT'].transform(np.max)
df_pos_cash_balance['K_CNT_INSTALMENT_MIN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT'].transform(np.min)

df_pos_cash_balance['K_CNT_INSTALMENT_FUTURE_MEAN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].transform(np.mean)
df_pos_cash_balance['K_CNT_INSTALMENT_FUTURE_MAX'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].transform(np.max)
# df_pos_cash_balance['K_CNT_INSTALMENT_FUTURE_MIN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].transform(np.min)

# #Drop Duplicates
df_pos_cash_balance = df_pos_cash_balance[['SK_ID_CURR', 'K_PREV_POS_CASH_BALANCE_COUNT', 
                                           'K_MONTHS_BALANCE_POS_CASH_MEAN','K_MONTHS_BALANCE_POS_CASH_MAX', 'K_MONTHS_BALANCE_POS_CASH_MIN', 
                                           'K_CNT_INSTALMENT_MEAN', 'K_CNT_INSTALMENT_MAX', 'K_CNT_INSTALMENT_MIN', 
                                           'K_CNT_INSTALMENT_FUTURE_MEAN', 'K_CNT_INSTALMENT_FUTURE_MAX']].drop_duplicates()

## Feature: df_previous_application

In [928]:
# #Missing values have been masked with the value 365243.0
df_previous_application['DAYS_LAST_DUE'].replace(365243.0, np.nan, inplace=True)
df_previous_application['DAYS_TERMINATION'].replace(365243.0, np.nan, inplace=True)
df_previous_application['DAYS_FIRST_DRAWING'].replace(365243.0, np.nan, inplace=True)
df_previous_application['DAYS_FIRST_DUE'].replace(365243.0, np.nan, inplace=True)
df_previous_application['DAYS_LAST_DUE_1ST_VERSION'].replace(365243.0, np.nan, inplace=True)

In [929]:
df_previous_application['K_PREV_PREVIOUS_APPLICATION_COUNT'] = df_previous_application.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')

df_previous_application['K_AMT_ANNUITY_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.mean)
df_previous_application['K_AMT_ANNUITY_MAX'] = df_previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.max)
df_previous_application['K_AMT_ANNUITY_MIN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.min)
df_previous_application['K_AMT_ANNUITY_VAR'] = df_previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.var)
# df_previous_application['K_AMT_ANNUITY_SUM'] = df_previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.sum)
# df_previous_application['K_AMT_ANNUITY_RANGE'] = df_previous_application['K_AMT_ANNUITY_MAX'] - df_previous_application['K_AMT_ANNUITY_MIN']

# #Features reduced the accuracy of my model
# df_previous_application['K_AMT_APPLICATION_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_APPLICATION'].transform(np.mean)
# df_previous_application['K_AMT_APPLICATION_MAX'] = df_previous_application.groupby('SK_ID_CURR')['AMT_APPLICATION'].transform(np.max)
# df_previous_application['K_AMT_APPLICATION_MIN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_APPLICATION'].transform(np.min)
# df_previous_application['K_AMT_APPLICATION_VAR'] = df_previous_application.groupby('SK_ID_CURR')['AMT_APPLICATION'].transform(np.var)
# df_previous_application['K_AMT_APPLICATION_SUM'] = df_previous_application.groupby('SK_ID_CURR')['AMT_APPLICATION'].transform(np.sum)

# df_previous_application['K_AMT_CREDIT_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_CREDIT'].transform(np.mean)
# df_previous_application['K_AMT_CREDIT_MAX'] = df_previous_application.groupby('SK_ID_CURR')['AMT_CREDIT'].transform(np.max)
# df_previous_application['K_AMT_CREDIT_MIN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_CREDIT'].transform(np.min)
# df_previous_application['K_AMT_CREDIT_VAR'] = df_previous_application.groupby('SK_ID_CURR')['AMT_CREDIT'].transform(np.var)
# df_previous_application['K_AMT_CREDIT_SUM'] = df_previous_application.groupby('SK_ID_CURR')['AMT_CREDIT'].transform(np.sum)

df_previous_application['TEMP_CREDIT_ALLOCATED'] = df_previous_application['AMT_CREDIT']/df_previous_application['AMT_APPLICATION']
df_previous_application['K_CREDIT_ALLOCATED_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['TEMP_CREDIT_ALLOCATED'].transform(np.mean)
df_previous_application['K_CREDIT_ALLOCATED_MAX'] = df_previous_application.groupby('SK_ID_CURR')['TEMP_CREDIT_ALLOCATED'].transform(np.max)
df_previous_application['K_CREDIT_ALLOCATED_MIN'] = df_previous_application.groupby('SK_ID_CURR')['TEMP_CREDIT_ALLOCATED'].transform(np.min)

df_previous_application['TEMP_NAME_CONTRACT_STATUS_APPROVED'] = (df_previous_application['NAME_CONTRACT_STATUS'] == 'Approved').astype(int)
df_previous_application['K_NAME_CONTRACT_STATUS_APPROVED'] = df_previous_application.groupby('SK_ID_CURR')['TEMP_NAME_CONTRACT_STATUS_APPROVED'].transform(np.sum)
# df_previous_application['TEMP_NAME_CONTRACT_STATUS_REFUSED'] = (df_previous_application['NAME_CONTRACT_STATUS'] == 'REFUSED').astype(int)
# df_previous_application['K_NAME_CONTRACT_STATUS_REFUSED'] = df_previous_application.groupby('SK_ID_CURR')['TEMP_NAME_CONTRACT_STATUS_REFUSED'].transform(np.sum)

df_previous_application['K_AMT_DOWN_PAYMENT_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_DOWN_PAYMENT'].transform(np.mean)
df_previous_application['K_AMT_DOWN_PAYMENT_MAX'] = df_previous_application.groupby('SK_ID_CURR')['AMT_DOWN_PAYMENT'].transform(np.max)
df_previous_application['K_AMT_DOWN_PAYMENT_MIN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_DOWN_PAYMENT'].transform(np.min)

df_previous_application['K_AMT_GOODS_PRICE_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_GOODS_PRICE'].transform(np.mean)
df_previous_application['K_AMT_GOODS_PRICE_MAX'] = df_previous_application.groupby('SK_ID_CURR')['AMT_GOODS_PRICE'].transform(np.max)
df_previous_application['K_AMT_GOODS_PRICE_MIN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_GOODS_PRICE'].transform(np.min)

df_previous_application['K_DAYS_DECISION_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_DECISION'].transform(np.mean)
df_previous_application['K_DAYS_DECISION_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_DECISION'].transform(np.max)
df_previous_application['K_DAYS_DECISION_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_DECISION'].transform(np.min)

df_previous_application['K_CNT_PAYMENT_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['CNT_PAYMENT'].transform(np.mean)
df_previous_application['K_CNT_PAYMENT_MAX'] = df_previous_application.groupby('SK_ID_CURR')['CNT_PAYMENT'].transform(np.max)
df_previous_application['K_CNT_PAYMENT_MIN'] = df_previous_application.groupby('SK_ID_CURR')['CNT_PAYMENT'].transform(np.min)

df_previous_application['K_DAYS_FIRST_DRAWING_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DRAWING'].transform(np.mean)
# df_previous_application['K_DAYS_FIRST_DRAWING_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DRAWING'].transform(np.max)
# df_previous_application['K_DAYS_FIRST_DRAWING_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DRAWING'].transform(np.min)

df_previous_application['K_DAYS_FIRST_DUE_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DUE'].transform(np.mean)
df_previous_application['K_DAYS_FIRST_DUE_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DUE'].transform(np.max)
df_previous_application['K_DAYS_FIRST_DUE_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DUE'].transform(np.min)

df_previous_application['K_DAYS_LAST_DUE_1ST_VERSION_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE_1ST_VERSION'].transform(np.mean)
df_previous_application['K_DAYS_LAST_DUE_1ST_VERSION_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE_1ST_VERSION'].transform(np.max)
df_previous_application['K_DAYS_LAST_DUE_1ST_VERSION_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE_1ST_VERSION'].transform(np.min)

df_previous_application['K_DAYS_LAST_DUE_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE'].transform(np.mean)
df_previous_application['K_DAYS_LAST_DUE_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE'].transform(np.max)
df_previous_application['K_DAYS_LAST_DUE_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE'].transform(np.min)

df_previous_application['K_DAYS_TERMINATION_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_TERMINATION'].transform(np.mean)
df_previous_application['K_DAYS_TERMINATION_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_TERMINATION'].transform(np.max)
df_previous_application['K_DAYS_TERMINATION_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_TERMINATION'].transform(np.min)


# #Drop Duplicates
df_previous_application = df_previous_application[['SK_ID_CURR', 'K_PREV_PREVIOUS_APPLICATION_COUNT', 
                                                   'K_NAME_CONTRACT_STATUS_APPROVED',
                                                   'K_AMT_ANNUITY_VAR', 
                                          'K_AMT_ANNUITY_MEAN', 'K_AMT_ANNUITY_MAX', 'K_AMT_ANNUITY_MIN',
                                          'K_CREDIT_ALLOCATED_MEAN', 'K_CREDIT_ALLOCATED_MAX', 'K_CREDIT_ALLOCATED_MIN',
                                          'K_AMT_DOWN_PAYMENT_MEAN', 'K_AMT_DOWN_PAYMENT_MAX', 'K_AMT_DOWN_PAYMENT_MIN',
                                          'K_AMT_GOODS_PRICE_MEAN', 'K_AMT_GOODS_PRICE_MAX', 'K_AMT_GOODS_PRICE_MIN',
                                          'K_DAYS_DECISION_MEAN', 'K_DAYS_DECISION_MAX', 'K_DAYS_DECISION_MIN',
                                          'K_CNT_PAYMENT_MEAN', 'K_CNT_PAYMENT_MAX', 'K_CNT_PAYMENT_MIN',
                                          'K_DAYS_FIRST_DRAWING_MEAN',
                                          'K_DAYS_FIRST_DUE_MEAN', 'K_DAYS_FIRST_DUE_MAX', 'K_DAYS_FIRST_DUE_MIN',
                                          'K_DAYS_LAST_DUE_1ST_VERSION_MEAN', 'K_DAYS_LAST_DUE_1ST_VERSION_MAX', 'K_DAYS_LAST_DUE_1ST_VERSION_MIN',
                                          'K_DAYS_LAST_DUE_MEAN', 'K_DAYS_LAST_DUE_MAX', 'K_DAYS_LAST_DUE_MIN',
                                          'K_DAYS_TERMINATION_MEAN', 'K_DAYS_TERMINATION_MAX', 'K_DAYS_TERMINATION_MIN']].drop_duplicates()

In [930]:
df_previous_application_original[df_previous_application_original['SK_ID_CURR'] == 100006]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
98162,2078043,100006,Cash loans,24246.00,675000.00,675000.0,NaN,675000.00,THURSDAY,15,Y,1,NaN,NaN,NaN,XNA,Approved,-181,Cash through the bank,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,48.0,low_normal,Cash X-Sell: low,365243.0,-151.0,1259.0,-151.0,-143.0,0.0
617859,2827850,100006,Revolving loans,NaN,0.00,0.0,NaN,NaN,THURSDAY,15,Y,1,NaN,NaN,NaN,XAP,Canceled,-181,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Card Street,NaN,NaN,NaN,NaN,NaN,NaN
830967,2190416,100006,Consumer loans,29027.52,334917.00,267930.0,66987.00,334917.00,SUNDAY,15,Y,1,0.217830,NaN,NaN,XAP,Approved,-311,Cash through the bank,XAP,Family,Repeater,Audio/Video,POS,XNA,Country-wide,8025,Consumer electronics,12.0,high,POS household with interest,365243.0,-281.0,49.0,365243.0,365243.0,0.0
900957,1489396,100006,Revolving loans,13500.00,270000.00,270000.0,NaN,270000.00,THURSDAY,15,Y,1,NaN,NaN,NaN,XAP,Approved,-181,XNA,XAP,Unaccompanied,Repeater,XNA,Cards,x-sell,Credit and cash offices,-1,XNA,0.0,XNA,Card X-Sell,365243.0,365243.0,365243.0,365243.0,365243.0,0.0
1131133,1020698,100006,Cash loans,39954.51,454500.00,481495.5,NaN,454500.00,SATURDAY,12,Y,1,NaN,NaN,NaN,XNA,Approved,-438,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,18.0,high,Cash X-Sell: high,NaN,NaN,NaN,NaN,NaN,NaN
1232752,1243599,100006,Cash loans,NaN,0.00,0.0,NaN,NaN,THURSDAY,15,Y,1,NaN,NaN,NaN,XNA,Canceled,-181,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
1333657,2299329,100006,Consumer loans,2482.92,26912.34,24219.0,2693.34,26912.34,TUESDAY,15,Y,1,0.108994,NaN,NaN,XAP,Approved,-617,XNA,XAP,NaN,New,Construction Materials,POS,XNA,Stone,30,Construction,12.0,middle,POS industry with interest,365243.0,-545.0,-215.0,-425.0,-416.0,0.0
1595430,2545789,100006,Cash loans,NaN,0.00,0.0,NaN,NaN,THURSDAY,15,Y,1,NaN,NaN,NaN,XNA,Canceled,-181,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
1607443,1697039,100006,Cash loans,32696.10,688500.00,906615.0,NaN,688500.00,THURSDAY,15,Y,1,NaN,NaN,NaN,XNA,Refused,-181,Cash through the bank,LIMIT,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,48.0,low_normal,Cash X-Sell: low,NaN,NaN,NaN,NaN,NaN,NaN


In [931]:
prev_app_df = df_previous_application_original.copy()

In [932]:
agg_funs = {'SK_ID_CURR': 'count', 'AMT_CREDIT': 'sum'}
prev_apps = prev_app_df.groupby('SK_ID_CURR').agg(agg_funs)
# prev_apps.columns = ['PREV APP COUNT', 'TOTAL PREV LOAN AMT']
# merged_df = app_data.merge(prev_apps, left_on='SK_ID_CURR', right_index=True, how='left')

In [933]:
prev_apps

,SK_ID_CURR,AMT_CREDIT
SK_ID_CURR,,
100001,1,23787.000
100002,1,179055.000
100003,3,1452573.000
100004,1,20106.000
100005,2,40153.500
100006,9,2625259.500
100007,6,999832.500
100008,5,813838.500
100009,7,490963.500


In [934]:
gc.collect()

117

In [959]:
# df_bureau = pd.read_csv("../data/bureau.csv")
# df_bureau_balance = pd.read_csv("../data/bureau_balance.csv")

## Feature: df_bureau_balance

In [960]:
# df_bureau_balance['K_BUREAU_BALANCE_COUNT'] = df_bureau_balance.groupby('SK_ID_BUREAU')['SK_ID_BUREAU'].transform('count')

df_bureau_balance['K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN'] = df_bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].transform(np.mean)
df_bureau_balance['K_BUREAU_BALANCE_MONTHS_BALANCE_MAX'] = df_bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].transform(np.max)
df_bureau_balance['K_BUREAU_BALANCE_MONTHS_BALANCE_MIN'] = df_bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].transform(np.min)

# df_bureau_balance[['K_BB_STATUS_0', 'K_BB_STATUS_1', 'K_BB_STATUS_2', 'K_BB_STATUS_3', 
#                    'K_BB_STATUS_4', 'K_BB_STATUS_5', 'K_BB_STATUS_C', 'K_BB_STATUS_X']] = pd.get_dummies(df_bureau_balance['STATUS'], prefix="K_BB_STATUS_")

# #Drop Duplicates
df_bureau_balance = df_bureau_balance[['SK_ID_BUREAU','K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN',
                                                 'K_BUREAU_BALANCE_MONTHS_BALANCE_MAX',
                                                'K_BUREAU_BALANCE_MONTHS_BALANCE_MIN']].drop_duplicates()

In [961]:
gc.collect()

105

## Feature: df_bureau

In [962]:
len(df_bureau["SK_ID_BUREAU"].unique())

1716428

In [963]:
df_bureau.shape

(1716428, 17)

In [964]:
df_bureau = pd.merge(df_bureau, df_bureau_balance, on="SK_ID_BUREAU", how="left", suffixes=('_bureau', '_bureau_balance'))

In [965]:
df_bureau.shape

(1716428, 20)

In [966]:
# #Feature - SK_ID_BUREAU represents each loan application.
# #Grouping by SK_ID_CURR significes the number of previous loans per applicant.
df_bureau['K_BUREAU_COUNT'] = df_bureau.groupby('SK_ID_CURR')['SK_ID_BUREAU'].transform('count')

# # #Feature - CREDIT_ACTIVE
# #Frequency Encoding
# temp_bureau_credit_active = df_bureau.groupby(['SK_ID_CURR','CREDIT_ACTIVE']).size()/df_bureau.groupby(['SK_ID_CURR']).size()
# temp_bureau_credit_active = temp_bureau_credit_active.to_frame().reset_index().rename(columns= {0: 'TEMP_BUREAU_CREDIT_ACTIVE_FREQENCODE'})
# temp_bureau_credit_active = temp_bureau_credit_active.pivot(index='SK_ID_CURR', columns='CREDIT_ACTIVE', values='TEMP_BUREAU_CREDIT_ACTIVE_FREQENCODE')
# temp_bureau_credit_active.reset_index(inplace = True)
# temp_bureau_credit_active.columns = ['SK_ID_CURR', 'K_CREDIT_ACTIVE_ACTIVE', 'K_CREDIT_ACTIVE_BADDEBT', 'K_CREDIT_ACTIVE_CLOSED', 'K_CREDIT_ACTIVE_SOLD']
# df_bureau = pd.merge(df_bureau, temp_bureau_credit_active, on=["SK_ID_CURR"], how="left", suffixes=('_bureau', '_credit_active_percentage'))
# del temp_bureau_credit_active

# #SUBSET DATA - CREDIT_ACTIVE == ACTIVE
# df_bureau_active = df_bureau[df_bureau['CREDIT_ACTIVE']=='Active']

# df_bureau_active['K_BUREAU_ACTIVE_DAYS_CREDIT_MEAN'] = df_bureau_active.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(np.mean)
# df_bureau_active['K_BUREAU_ACTIVE_DAYS_CREDIT_MAX'] = df_bureau_active.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(np.max)
# df_bureau = pd.merge(df_bureau, df_bureau_active, on=["SK_ID_CURR"], how="left", suffixes=('_bureau', '_bureau_active'))


# # #Feature - DAYS_CREDIT
df_bureau['K_BUREAU_DAYS_CREDIT_MEAN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(np.mean)
df_bureau['K_BUREAU_DAYS_CREDIT_MAX'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(np.max)
df_bureau['K_BUREAU_DAYS_CREDIT_MIN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(np.min)
# df_bureau['K_BUREAU_DAYS_CREDIT_VAR'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(np.var)

# # #Feature - DAYS_CREDIT_UPDATE
# df_bureau['K_BUREAU_DAYS_CREDIT_UPDATE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_UPDATE'].transform(np.mean)
# df_bureau['K_BUREAU_DAYS_CREDIT_UPDATE_MAX'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_UPDATE'].transform(np.max)
# df_bureau['K_BUREAU_DAYS_CREDIT_UPDATE_MIN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_UPDATE'].transform(np.min)
# df_bureau['K_BUREAU_DAYS_CREDIT_UPDATE_VAR'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_UPDATE'].transform(np.var)

gc.collect()
# # #Successive difference between credit application per customer - Mean, Min, Max
temp_bureau_days_credit = df_bureau.copy()
temp_bureau_days_credit.sort_values(['SK_ID_CURR', 'DAYS_CREDIT'], inplace=True)
temp_bureau_days_credit['temp_successive_diff'] = temp_bureau_days_credit.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(lambda ele: ele.diff())
temp_bureau_days_credit['K_BUREAU_DAYS_CREDIT_SORTED_SUCCESSIVE_DIFF_MEAN'] = temp_bureau_days_credit.groupby('SK_ID_CURR')['temp_successive_diff'].transform(np.mean)
df_bureau = pd.merge(df_bureau, temp_bureau_days_credit[['SK_ID_CURR','K_BUREAU_DAYS_CREDIT_SORTED_SUCCESSIVE_DIFF_MEAN']].drop_duplicates(), 
                     on="SK_ID_CURR", how="left", suffixes=('_bureau', '_days_credit_sorted_successive_diff'))
# del temp_bureau_days_credit

# df_bureau['K_BUREAU_CREDIT_DAY_OVERDUE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].transform(np.mean)
# df_bureau['K_BUREAU_CREDIT_DAY_OVERDUE_MAX'] = df_bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].transform(np.max)
# df_bureau['K_BUREAU_CREDIT_DAY_OVERDUE_MIN'] = df_bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].transform(np.min)

df_bureau['K_BUREAU_DAYS_CREDIT_ENDDATE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].transform(np.mean)
df_bureau['K_BUREAU_DAYS_CREDIT_ENDDATE_MAX'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].transform(np.max)
df_bureau['K_BUREAU_DAYS_CREDIT_ENDDATE_MIN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].transform(np.min)

df_bureau['K_BUREAU_DAYS_ENDDATE_FACT_MEAN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_ENDDATE_FACT'].transform(np.mean)
df_bureau['K_BUREAU_DAYS_ENDDATE_FACT_MAX'] = df_bureau.groupby('SK_ID_CURR')['DAYS_ENDDATE_FACT'].transform(np.max)
df_bureau['K_BUREAU_DAYS_ENDDATE_FACT_MIN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_ENDDATE_FACT'].transform(np.min)

df_bureau['K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].transform(np.min)


# df_bureau['K_BUREAU_CNT_CREDIT_PROLONG_MAX'] = df_bureau.groupby('SK_ID_CURR')['CNT_CREDIT_PROLONG'].transform(np.max)
# df_bureau['K_BUREAU_CNT_CREDIT_PROLONG_SUM'] = df_bureau.groupby('SK_ID_CURR')['CNT_CREDIT_PROLONG'].transform(np.sum)

# #To-Do: Calculate a utilization metric for some of the features below?
df_bureau['K_BUREAU_AMT_CREDIT_SUM_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform(np.min)

df_bureau['K_BUREAU_AMT_CREDIT_SUM_DEBT_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_DEBT_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_DEBT_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].transform(np.min)

df_bureau['K_BUREAU_AMT_CREDIT_SUM_LIMIT_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_LIMIT'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_LIMIT_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_LIMIT'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_LIMIT_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_LIMIT'].transform(np.min)

df_bureau['K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].transform(np.min)

df_bureau['K_BUREAU_AMT_ANNUITY_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.mean)
df_bureau['K_BUREAU_AMT_ANNUITY_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.max)
df_bureau['K_BUREAU_AMT_ANNUITY_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.min)


# #Added from df_bureau_balance
df_bureau['K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN'].transform(np.mean)
df_bureau['K_BUREAU_BALANCE_MONTHS_BALANCE_MAX'] = df_bureau.groupby('SK_ID_CURR')['K_BUREAU_BALANCE_MONTHS_BALANCE_MAX'].transform(np.max)
df_bureau['K_BUREAU_BALANCE_MONTHS_BALANCE_MIN'] = df_bureau.groupby('SK_ID_CURR')['K_BUREAU_BALANCE_MONTHS_BALANCE_MIN'].transform(np.min)



#Drop Duplicates
df_bureau = df_bureau[['SK_ID_CURR', 'K_BUREAU_COUNT', 
                       'K_BUREAU_DAYS_CREDIT_MEAN', 'K_BUREAU_DAYS_CREDIT_MAX', 'K_BUREAU_DAYS_CREDIT_MIN',
                       'K_BUREAU_DAYS_CREDIT_SORTED_SUCCESSIVE_DIFF_MEAN',
                      'K_BUREAU_DAYS_CREDIT_ENDDATE_MEAN', 'K_BUREAU_DAYS_CREDIT_ENDDATE_MAX', 'K_BUREAU_DAYS_CREDIT_ENDDATE_MIN',
                      'K_BUREAU_DAYS_ENDDATE_FACT_MEAN', 'K_BUREAU_DAYS_ENDDATE_FACT_MAX', 'K_BUREAU_DAYS_ENDDATE_FACT_MIN',
                      'K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN', 'K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX', 'K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MIN',
                      'K_BUREAU_AMT_CREDIT_SUM_MEAN', 'K_BUREAU_AMT_CREDIT_SUM_MAX', 'K_BUREAU_AMT_CREDIT_SUM_MIN',
                      'K_BUREAU_AMT_CREDIT_SUM_DEBT_MEAN', 'K_BUREAU_AMT_CREDIT_SUM_DEBT_MAX', 'K_BUREAU_AMT_CREDIT_SUM_DEBT_MIN',
                      'K_BUREAU_AMT_CREDIT_SUM_LIMIT_MEAN', 'K_BUREAU_AMT_CREDIT_SUM_LIMIT_MAX', 'K_BUREAU_AMT_CREDIT_SUM_LIMIT_MIN',
                      'K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MEAN', 'K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MAX', 'K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MIN',
                      'K_BUREAU_AMT_ANNUITY_MEAN', 'K_BUREAU_AMT_ANNUITY_MAX', 'K_BUREAU_AMT_ANNUITY_MIN',
                      'K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN', 'K_BUREAU_BALANCE_MONTHS_BALANCE_MAX', 'K_BUREAU_BALANCE_MONTHS_BALANCE_MIN']].drop_duplicates()

In [967]:
df_bureau.shape

(305811, 33)

In [968]:
# #CHECKPOINT
print("df_bureau_original", df_bureau_original.shape)
print(len(set(df_bureau_original["SK_ID_CURR"]).intersection(set(df_application_train["SK_ID_CURR"]))))
print(len(set(df_bureau_original["SK_ID_CURR"]).intersection(set(df_application_test["SK_ID_CURR"]))))
print("Sum: ", 263491 + 42320)

df_bureau_original (1716428, 17)
263491
42320
Sum:  305811


In [969]:
# #CHECKPOINT
print("df_bureau", df_bureau.shape)
print(len(set(df_bureau["SK_ID_CURR"]).intersection(set(df_application_train["SK_ID_CURR"]))))
print(len(set(df_bureau["SK_ID_CURR"]).intersection(set(df_application_test["SK_ID_CURR"]))))
print("Sum: ", 263491 + 42320)

df_bureau (305811, 33)
263491
42320
Sum:  305811


In [970]:
gc.collect()

43

## Feature MAIN TABLE: df_application_train

In [ ]:
# #Missing values have been masked with the value 365243
df_application_train['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
df_application_test['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

In [ ]:
# #Feature - Divide existing features
df_application_train['K_APP_CREDIT_TO_INCOME_RATIO'] = df_application_train['AMT_CREDIT']/df_application_train['AMT_INCOME_TOTAL']
df_application_train['K_APP_ANNUITY_TO_INCOME_RATIO'] = df_application_train['AMT_ANNUITY']/df_application_train['AMT_INCOME_TOTAL']
df_application_train['K_APP_CREDIT_TO_ANNUITY_RATIO'] = df_application_train['AMT_CREDIT']/df_application_train['AMT_ANNUITY']
# df_application_train['K_APP_INCOME_PER_PERSON_RATIO'] = df_application_train['AMT_INCOME_TOTAL'] / df_application_train['CNT_FAM_MEMBERS']

df_application_train['K_APP_CREDITTOINCOME_TO_DAYSEMPLOYED_RATIO'] = df_application_train['K_APP_CREDIT_TO_INCOME_RATIO'] /df_application_train['DAYS_EMPLOYED']
df_application_train['K_APP_ANNUITYTOINCOME_TO_DAYSEMPLOYED_RATIO'] = df_application_train['K_APP_ANNUITY_TO_INCOME_RATIO'] /df_application_train['DAYS_EMPLOYED']
df_application_train['K_APP_ANNUITYTOCREDIT_TO_DAYSEMPLOYED_RATIO'] = df_application_train['K_APP_CREDIT_TO_ANNUITY_RATIO'] /df_application_train['DAYS_EMPLOYED']


df_application_train['K_APP_CREDIT_TO_GOODSPRICE_RATIO'] = df_application_train['AMT_CREDIT']/df_application_train['AMT_GOODS_PRICE']
df_application_train['K_APP_GOODSPRICE_TO_INCOME_RATIO'] = df_application_train['AMT_GOODS_PRICE']/df_application_train['AMT_INCOME_TOTAL']
df_application_train['K_APP_GOODSPRICE_TO_ANNUITY_RATIO'] = df_application_train['AMT_GOODS_PRICE']/df_application_train['AMT_ANNUITY']

# #Feature - Income, Education, Family Status
df_application_train['K_APP_INCOME_EDUCATION'] = df_application_train['NAME_INCOME_TYPE'] + df_application_train['NAME_EDUCATION_TYPE']
df_application_train['K_APP_INCOME_EDUCATION_FAMILY'] = df_application_train['NAME_INCOME_TYPE'] + df_application_train['NAME_EDUCATION_TYPE'] + df_application_train['NAME_FAMILY_STATUS']

# #Family
df_application_train['K_APP_EMPLOYED_TO_DAYSBIRTH_RATIO'] = df_application_train['DAYS_EMPLOYED']/df_application_train['DAYS_BIRTH']
df_application_train['K_APP_INCOME_PER_FAMILY'] = df_application_train['AMT_INCOME_TOTAL']/df_application_train['CNT_FAM_MEMBERS']
# df_application_train['K_APP_CHILDREN_TO_FAMILY_RATIO'] = df_application_train['CNT_CHILDREN']/df_application_train['CNT_FAM_MEMBERS']

# df_application_train['K_APP_FLAGS_SUM'] = df_application_train.loc[:, df_application_train.columns.str.contains('FLAG_DOCUMENT')].sum(axis=1)
    
df_application_train['K_APP_EXT_SOURCES_MEAN'] = df_application_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)

In [ ]:
# #Feature - Divide existing features
df_application_test['K_APP_CREDIT_TO_INCOME_RATIO'] = df_application_test['AMT_CREDIT']/df_application_test['AMT_INCOME_TOTAL']
df_application_test['K_APP_ANNUITY_TO_INCOME_RATIO'] = df_application_test['AMT_ANNUITY']/df_application_test['AMT_INCOME_TOTAL']
df_application_test['K_APP_CREDIT_TO_ANNUITY_RATIO'] = df_application_test['AMT_CREDIT']/df_application_test['AMT_ANNUITY']
# df_application_test['K_APP_INCOME_PER_PERSON_RATIO'] = df_application_test['AMT_INCOME_TOTAL'] / df_application_test['CNT_FAM_MEMBERS']

df_application_test['K_APP_CREDITTOINCOME_TO_DAYSEMPLOYED_RATIO'] = df_application_test['K_APP_CREDIT_TO_INCOME_RATIO'] /df_application_test['DAYS_EMPLOYED']
df_application_test['K_APP_ANNUITYTOINCOME_TO_DAYSEMPLOYED_RATIO'] = df_application_test['K_APP_ANNUITY_TO_INCOME_RATIO'] /df_application_test['DAYS_EMPLOYED']
df_application_test['K_APP_ANNUITYTOCREDIT_TO_DAYSEMPLOYED_RATIO'] = df_application_test['K_APP_CREDIT_TO_ANNUITY_RATIO'] /df_application_test['DAYS_EMPLOYED']


df_application_test['K_APP_CREDIT_TO_GOODSPRICE_RATIO'] = df_application_test['AMT_CREDIT']/df_application_test['AMT_GOODS_PRICE']
df_application_test['K_APP_GOODSPRICE_TO_INCOME_RATIO'] = df_application_test['AMT_GOODS_PRICE']/df_application_test['AMT_INCOME_TOTAL']
df_application_test['K_APP_GOODSPRICE_TO_ANNUITY_RATIO'] = df_application_test['AMT_GOODS_PRICE']/df_application_test['AMT_ANNUITY']

# #Feature - Income, Education, Family Status
df_application_test['K_APP_INCOME_EDUCATION'] = df_application_test['NAME_INCOME_TYPE'] + df_application_test['NAME_EDUCATION_TYPE']
df_application_test['K_APP_INCOME_EDUCATION_FAMILY'] = df_application_test['NAME_INCOME_TYPE'] + df_application_test['NAME_EDUCATION_TYPE'] + df_application_test['NAME_FAMILY_STATUS']


# #Family
df_application_test['K_APP_EMPLOYED_TO_DAYSBIRTH_RATIO'] = df_application_test['DAYS_EMPLOYED']/df_application_test['DAYS_BIRTH']
df_application_test['K_APP_INCOME_PER_FAMILY'] = df_application_test['AMT_INCOME_TOTAL']/df_application_test['CNT_FAM_MEMBERS']
# df_application_test['K_APP_CHILDREN_TO_FAMILY_RATIO'] = df_application_test['CNT_CHILDREN']/df_application_test['CNT_FAM_MEMBERS']

# df_application_test['K_APP_FLAGS_SUM'] = df_application_test.loc[:, df_application_test.columns.str.contains('FLAG_DOCUMENT')].sum(axis=1)

df_application_test['K_APP_EXT_SOURCES_MEAN'] = df_application_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)

In [ ]:
df_application_train.drop(['FLAG_MOBIL', 'APARTMENTS_AVG'], axis=1, inplace=True)
df_application_test.drop(['FLAG_MOBIL', 'APARTMENTS_AVG'], axis=1, inplace=True)

In [ ]:
gc.collect()

# Combine Datasets

## Encode categorical columns

In [ ]:
arr_categorical_columns = df_application_train.select_dtypes(['object']).columns
for var_col in arr_categorical_columns:
    df_application_train[var_col] = df_application_train[var_col].astype('category').cat.codes
gc.collect()

arr_categorical_columns = df_application_test.select_dtypes(['object']).columns
for var_col in arr_categorical_columns:
    df_application_test[var_col] = df_application_test[var_col].astype('category').cat.codes
gc.collect()

# arr_categorical_columns = df_credit_card_balance.select_dtypes(['object']).columns
# for var_col in arr_categorical_columns:
#     df_credit_card_balance[var_col] = df_credit_card_balance[var_col].astype('category').cat.codes

In [ ]:
# # One-hot encoding for categorical columns with get_dummies
# def one_hot_encoder(df, nan_as_category = True):
#     original_columns = list(df.columns)
#     categorical_columns = df.select_dtypes(['object']).columns
#     df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
#     new_columns = [c for c in df.columns if c not in original_columns]
#     return df, new_columns



## Combine Datasets

### df_installments_payments

In [ ]:
df_installments_payments_train = df_installments_payments[df_installments_payments["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_installments_payments_test = df_installments_payments[df_installments_payments["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [ ]:
df_application_train = pd.merge(df_application_train, df_installments_payments_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_installments_payments'))
df_application_test = pd.merge(df_application_test, df_installments_payments_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_installments_payments'))

### df_credit_card_balance

In [ ]:
df_credit_card_balance_train = df_credit_card_balance[df_credit_card_balance["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_credit_card_balance_test = df_credit_card_balance[df_credit_card_balance["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [ ]:
df_application_train = pd.merge(df_application_train, df_credit_card_balance_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_credit_card_balance'))
df_application_test = pd.merge(df_application_test, df_credit_card_balance_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_credit_card_balance'))

### df_pos_cash_balance

In [ ]:
df_pos_cash_balance_train = df_pos_cash_balance[df_pos_cash_balance["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_pos_cash_balance_test = df_pos_cash_balance[df_pos_cash_balance["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [ ]:
df_application_train = pd.merge(df_application_train, df_pos_cash_balance_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_pos_cash_balance'))
df_application_test = pd.merge(df_application_test, df_pos_cash_balance_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_pos_cash_balance'))

### df_previous_application

In [ ]:
df_previous_application_train = df_previous_application[df_previous_application["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_previous_application_test = df_previous_application[df_previous_application["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [ ]:
df_application_train = pd.merge(df_application_train, df_previous_application_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_previous_application'))
df_application_test = pd.merge(df_application_test, df_previous_application_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_previous_application'))

### df_bureau_balance and df_bureau

In [ ]:
df_bureau_train = df_bureau[df_bureau["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_bureau_test = df_bureau[df_bureau["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [ ]:
df_application_train = pd.merge(df_application_train, df_bureau_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_bureau'))
df_application_test = pd.merge(df_application_test, df_bureau_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_bureau'))

In [ ]:
gc.collect()

# Model Building

## Train-Validation Split

In [ ]:
input_columns = df_application_train.columns
input_columns = input_columns[input_columns != 'TARGET']
target_column = 'TARGET'

X = df_application_train[input_columns]
y = df_application_train[target_column]
gc.collect()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.7)

In [ ]:
lgb_params = {
    'boosting_type': 'gbdt',
    'colsample_bytree': 0.7,               # #Same as feature_fraction
    'learning_rate': 0.1604387053222455,
    'max_depth' : 4,
    'metric' : 'auc',
    'min_child_weight': 9,
    'num_boost_round': 5000, ##??
    'nthread': -1,
    'objective': 'binary',
    'lambda_l1': 3.160842634951819,
    'lambda_l2': 4.438488456929287,
    'reg_gamma':0.6236454630290655,
    'seed': 42,
    'subsample': 0.8,               # #Same as bagging_fraction
    'verbose': 1,
    }
    
#    'max_bin': 100}
# 'num_leaves': 40, 
#task?
#num_boost_round
#can I run this on my gpu?

In [ ]:
lgb_params = {
    'boosting_type': 'gbdt',
    'colsample_bytree': 0.7,               # #Same as feature_fraction
    'learning_rate': 0.0204387053222455,
    'max_depth' : 4,
    'metric' : 'auc',
    'min_child_weight': 9,
    'num_boost_round': 5000, ##??
    'nthread': -1,
    'objective': 'binary',
    'lambda_l1': 3.160842634951819,
    'lambda_l2': 4.438488456929287,
    'reg_gamma':0.6236454630290655,
    'seed': 42,
    'subsample': 0.8,               # #Same as bagging_fraction
    'verbose': 1,
    }
    
#    'max_bin': 100}
# 'num_leaves': 40, 
#task?
#num_boost_round
#can I run this on my gpu?
            


In [ ]:
# dtrain_lgb = lgb.Dataset(X_train, label=y_train)
# dtest_lgb = lgb.Dataset(X_test, label=y_test)

dtrain_lgb = lgb.Dataset(X, label=y)

cv_result_lgb = lgb.cv(lgb_params, 
                       dtrain_lgb, 
                       num_boost_round=5000, 
                       nfold=5, 
                       stratified=True, 
                       early_stopping_rounds=200, 
                       verbose_eval=100, 
                       show_stdv=True)

num_boost_rounds_lgb = len(cv_result_lgb['auc-mean'])
print(max(cv_result_lgb['auc-mean']))
print('num_boost_rounds_lgb=' + str(num_boost_rounds_lgb))

In [ ]:
# #Final Model
gc.collect()
# model = lgb.train(lgbm_params, lgb.Dataset(X_train,label=y_train), 270, lgb.Dataset(X_test,label=y_test), verbose_eval= 50)
# model = lgb.train(lgbm_params, lgb.Dataset(X,y), 270, [lgb.Dataset(X_train,label=y_train), lgb.Dataset(X_test,label=y_test)],verbose_eval= 50)

model_lgb = lgb.train(lgb_params, lgb.Dataset(X, label=y), num_boost_round=num_boost_rounds_lgb, verbose_eval= 50)

In [ ]:
df_predict = model_lgb.predict(df_application_test, num_iteration=model_lgb.best_iteration)

In [ ]:
submission = pd.DataFrame()
submission["SK_ID_CURR"] =  df_application_test["SK_ID_CURR"]
submission["TARGET"] =  df_predict

submission.to_csv("../submissions/model_2_lightgbm_v41.csv", index=False)

In [ ]:
# #Should be 48744, 2
submission.shape

In [ ]:
# #Feature Importance
importance = dict(zip(X_train.columns, model_lgb.feature_importance()))

sorted(((value,key) for (key,value) in importance.items()), reverse=True)

In [ ]:
# #Total Number of Features --- 227
len(sorted(((value,key) for (key,value) in importance.items()), reverse=True))

In [ ]:
# #Number of Features > 50 --- 21
len(sorted(((value,key) for (key,value) in importance.items() if value > 50), reverse=True))

In [ ]:
# #Number of Features > 60 --- 21
# (sorted(((key) for (key,value) in importance.items() if value > 50), reverse=True))

In [ ]:
# #Custom Interrupt Point for Run All below condition
def

In [ ]:
def def 

In [ ]:
cv_metrics = """
[100]	cv_agg's auc: 0.781284 + 0.0026998
[200]	cv_agg's auc: 0.786374 + 0.00245158
[300]	cv_agg's auc: 0.787628 + 0.00275346
[400]	cv_agg's auc: 0.787835 + 0.00279943
num_boost_rounds_lgb=390
"""

In [ ]:
lb_metrics = 0.790

In [ ]:
mongo_json = {
    "model" : "lightgbm",
    "model_params" : lgb_params,
    "feature_importance" : dict([(key, int(val)) for key, val in importance.items()]),
    "cv_metrics": cv_metrics,
    "lb_metrics" : lb_metrics
}

# MongoDB - My Personal ModelDB

In [ ]:
client = MongoClient()
client = MongoClient('localhost', 27017)

db = client.kaggle_homecredit
collection = db.model2_lightgbm

In [ ]:
db = client.kaggle_homecredit
collection = db.model2_lightgbm

In [ ]:
collection.insert_one(mongo_json)

In [ ]:
df_application_train.size